In [35]:
import pandas as pd

In [47]:
df_it = pd.read_excel('C:\고급프로그래밍\crawling_data\linkareer_crawling_it_internet.xlsx')
df_prod = pd.read_csv('C:\고급프로그래밍\crawling_data\linkareer_crawling_Production.csv')
df_re = pd.read_csv('C:\고급프로그래밍\crawling_data\linkareer_crawling_research.csv')

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:3: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:3: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13368\301069450.py:1: SyntaxWarning: invalid escape sequence '\c'
  df_it = pd.read_excel('C:\고급프로그래밍\crawling_data\linkareer_crawling_it_internet.xlsx')
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13368\301069450.py:2: SyntaxWarning: invalid escape sequence '\c'
  df_prod = pd.read_csv('C:\고급프로그래밍\crawling_data\linkareer_crawling_Production.csv')
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13368\301069450.py:3: SyntaxWarning: invalid escape sequence '\c'
  df_re = pd.read_csv('C:\고급프로그래밍\crawling_data\linkareer_crawling_research.csv')


In [48]:
import pandas as pd
import re

def preprocess_qualification(df, col='qualification'):
    """
    채용공고의 qualification 텍스트를 정제 및 필드 추출하여 데이터프레임에 새 컬럼 추가
    Parameters:
        df : pandas.DataFrame - 입력 데이터프레임
        col : str - 자격 요건 텍스트가 담긴 열 이름 (기본값: 'qualification')
    Returns:
        df : pandas.DataFrame - 새로운 전처리 컬럼이 포함된 데이터프레임
    """
    df = df.copy()

    # 1. 기본 클렌징
    df[col] = df[col].fillna('').replace('#NAME?', '', regex=False)
    df['qualification_clean'] = df[col].apply(lambda x: re.sub(r'[\n\r•▪·ㆍ\-@]+', ' ', x))
    df['qualification_clean'] = df['qualification_clean'].str.strip()

    # 2. 필드 추출 함수 정의
    def extract_education(text):
        if '고졸' in text or '고등학교' in text:
            return '고졸'
        elif '전문학사' in text or '2년제' in text or '전문대' in text:
            return '전문학사'
        elif '학사' in text or '4년제' in text or '대졸' in text:
            return '학사'
        elif '석사' in text:
            return '석사'
        elif '박사' in text:
            return '박사'
        return None

    def extract_major(text):
        majors = ['전기', '전자', '컴퓨터', '화학', '기계', '소프트웨어', '산업', '반도체', '재료', '이공', '공학', '생명', '수학', '물리']
        for major in majors:
            if major in text:
                return major + ' 계열'
        return None

    def extract_experience(text):
        match = re.search(r'경력\s*:?[^0-9]*(\d+)\s*년', text)
        return int(match.group(1)) if match else 0

    def extract_language(text):
        if 'TOEIC' in text or 'OPIC' in text or '영어' in text:
            return '영어 성적 요구'
        return None

    def extract_military(text):
        if '병역' in text or '군필' in text:
            return '필수'
        return '무관'

    def extract_overseas(text):
        return '필수' if '해외여행에 결격사유' in text or '해외 출장' in text else '무관'

    # 3. 컬럼 생성
    df['education_level'] = df['qualification_clean'].apply(extract_education)
    df['major'] = df['qualification_clean'].apply(extract_major)
    df['experience_years'] = df['qualification_clean'].apply(extract_experience)
    df['language_requirement'] = df['qualification_clean'].apply(extract_language)
    df['military_requirement'] = df['qualification_clean'].apply(extract_military)
    df['overseas_available'] = df['qualification_clean'].apply(extract_overseas)

    # 기타 요구 사항 전체를 따로 저장
    df['etc_requirements'] = df['qualification_clean']

    return df

In [49]:
df_it = preprocess_qualification(df_it)
df_prod = preprocess_qualification(df_prod)
df_re = preprocess_qualification(df_re)

In [51]:
def prepare_for_db(df):
    """
    전처리 완료된 df를 DB에 넣기 위한 형태로 최종 정리
    - 결측값 처리
    - 데이터 타입 정제
    - 컬럼 순서 정리
    - 불필요한 컬럼 제거
    Returns:
        df : DB 저장 가능한 최종 정리된 DataFrame
    """
    df = df.copy()

    # 결측값 채우기
    df['education_level'] = df['education_level'].fillna('').astype(str)
    df['major'] = df['major'].fillna('').astype(str)
    df['language_requirement'] = df['language_requirement'].fillna('').astype(str)
    df['military_requirement'] = df['military_requirement'].fillna('').astype(str)
    df['overseas_available'] = df['overseas_available'].fillna('').astype(str)
    df['experience_years'] = df['experience_years'].fillna(0).astype(int)
    df['etc_requirements'] = df['etc_requirements'].fillna('').astype(str)

    # 기타 주요 컬럼도 안전하게 문자형으로 변환
    for col in ['company_name', 'company_type', 'main_job', 'detail_job', 'location', 'process']:
        if col in df.columns:
            df[col] = df[col].fillna('').astype(str)

    # 불필요한 중간 컬럼 제거
    drop_cols = ['qualification', 'qualification_clean']
    for col in drop_cols:
        if col in df.columns:
            df.drop(columns=col, inplace=True)

    # 열 순서 재정렬
    cols_order = ['company_name', 'company_type', 'main_job', 'detail_job', 'location',
                  'education_level', 'major', 'experience_years', 'language_requirement',
                  'military_requirement', 'overseas_available', 'etc_requirements', 'process', 'job_category']

    # 존재하는 열만 정렬
    df = df[[col for col in cols_order if col in df.columns]]

    return df

In [52]:
df_it = prepare_for_db(df_it)
df_prod = prepare_for_db(df_prod)
df_re = prepare_for_db(df_re)

In [53]:
df_re.head()

,company_name,company_type,main_job,detail_job,location,education_level,major,experience_years,language_requirement,military_requirement,overseas_available,etc_requirements,process,job_category
0,보쉬전장,중견기업,,,세종특별자치시,,,0,,무관,무관,,,연구개발/설계
1,성신양회,중견기업,충북 단양군,전기,,학사,전기 계열,5,,무관,무관,"학력 : 4년제졸 이상 전공 : 전기, 전기공학 계열 경력 신입 또는 경력 5년 이상",,연구개발/설계
2,SFA반도체,중견기업,,,충남 천안시 서북구,,,0,,무관,무관,,,연구개발/설계
3,큐알티,중소기업,,FIB,경기 수원시 영통구,전문학사,이공 계열,0,,무관,무관,(1) 학력: 전문학사 (2) 전공: 이공계열,,연구개발/설계
4,현대제철,대기업,,#NAME?,경북 포항시 남구,,,0,,무관,무관,,,연구개발/설계


In [54]:
df_it.head()

,company_name,company_type,main_job,detail_job,location,education_level,major,experience_years,language_requirement,military_requirement,overseas_available,etc_requirements,process,job_category
0,Deloitte 안진회계법인,외국계기업,,,서울 영등포구,,,0,,무관,무관,,,IT/인터넷
1,현대피엔에스,대기업,,세종R&D센터,세종특별자치시,학사,전기 계열,5,,무관,무관,경력 사항 대리 과장급 : 관련 직무 경력 5년 이상 사원급 : 관련 직무 경력...,[전형 안내]\n서류전형 - 면접전형 - 최종합격,IT/인터넷
2,한국동서발전,공공기관/공기업,,,"울산 중구, 충남 당진시, 강원특별자치도 동해시, 경기 고양시 일산동구, 대전 유성구",,,0,,무관,무관,,,IT/인터넷
3,카카오,대기업,,,경기 성남시 분당구,,,0,,무관,무관,,,IT/인터넷
4,대한지방행정공제회,비영리단체/협회/재단,"경영/사무, IT/인터넷",일반,,,,0,,무관,무관,,,IT/인터넷


In [56]:
df_it.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   company_name          384 non-null    object
 1   company_type          384 non-null    object
 2   main_job              384 non-null    object
 3   detail_job            384 non-null    object
 4   location              384 non-null    object
 5   education_level       384 non-null    object
 6   major                 384 non-null    object
 7   experience_years      384 non-null    int64 
 8   language_requirement  384 non-null    object
 9   military_requirement  384 non-null    object
 10  overseas_available    384 non-null    object
 11  etc_requirements      384 non-null    object
 12  process               384 non-null    object
 13  job_category          384 non-null    object
dtypes: int64(1), object(13)
memory usage: 42.1+ KB


In [57]:
df_prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   company_name          142 non-null    object
 1   company_type          142 non-null    object
 2   main_job              142 non-null    object
 3   detail_job            142 non-null    object
 4   location              142 non-null    object
 5   education_level       142 non-null    object
 6   major                 142 non-null    object
 7   experience_years      142 non-null    int64 
 8   language_requirement  142 non-null    object
 9   military_requirement  142 non-null    object
 10  overseas_available    142 non-null    object
 11  etc_requirements      142 non-null    object
 12  process               142 non-null    object
 13  job_category          142 non-null    object
dtypes: int64(1), object(13)
memory usage: 15.7+ KB


In [55]:
df_prod.head()

,company_name,company_type,main_job,detail_job,location,education_level,major,experience_years,language_requirement,military_requirement,overseas_available,etc_requirements,process,job_category
0,조선내화,대기업,,제조기술직_전남 광양,"전남 광양시, 경북 포항시 남구",고졸,,0,,필수,무관,"고등학교 이상 졸업인 분으로서, 2025년 7월 내 회사가 지정한 입사일에 입사 가...",,생산/제조
1,코오롱생명과학,대기업,,,충북 충주시,,,0,,무관,무관,,,생산/제조
2,Englewood LAB,기타,,,"생산/제조, 연구개발/설계",,,0,,무관,무관,,,생산/제조
3,포스코모빌리티솔루션,대기업,,,경북 포항시 남구,,,0,,무관,무관,,,생산/제조
4,크라운제과,중견기업,,일반관리,"서울 용산구, 대전 대덕구, 경기 양주시",학사,,0,,무관,무관,① 학력: 4년제 대졸학사 이상,,생산/제조


In [58]:
df_it.to_csv('C:\고급프로그래밍\crawling_data\linkareer_crawling_it_internet_preprocessed.csv', index=False, encoding='utf-8-sig')
df_prod.to_csv('C:\고급프로그래밍\crawling_data\linkareer_crawling_Production_preprocessed.csv', index=False, encoding='utf-8-sig')
df_re.to_csv('C:\고급프로그래밍\crawling_data\linkareer_crawling_research_preprocessed.csv', index=False, encoding='utf-8-sig')

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:3: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:3: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13368\3080012499.py:1: SyntaxWarning: invalid escape sequence '\c'
  df_it.to_csv('C:\고급프로그래밍\crawling_data\linkareer_crawling_it_internet_preprocessed.csv', index=False, encoding='utf-8-sig')
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13368\3080012499.py:2: SyntaxWarning: invalid escape sequence '\c'
  df_prod.to_csv('C:\고급프로그래밍\crawling_data\linkareer_crawling_Production_preprocessed.csv', index=False, encoding='utf-8-sig')
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13368\3080012499.py:3: SyntaxWarning: invalid escape sequence '\c'
  df_re.to_csv('C:\고급프로그래밍\crawling_data\linkareer_crawling_research_preprocessed.csv', index=False, encoding='